# Notebook 5: Tổng kết Toàn bộ Dự án

**Mục tiêu:** Tổng hợp tất cả kết quả và tạo summary cho báo cáo

**Nội dung:**
1. Load và tổng hợp kết quả từ tất cả notebooks
2. Tạo final summary statistics
3. Key visualizations cho báo cáo
4. Recommendations và conclusions
5. Export data cho báo cáo

## 1. Setup và Import

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

from image_utils import load_image, get_image_info
from svd_compression import compress_grayscale, compress_rgb, calculate_compression_ratio, calculate_cumulative_energy
from quality_metrics import calculate_all_metrics
from visualization import *

plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")
sns.set_palette("husl")

print("="*80)
print("FINAL SUMMARY - IMAGE COMPRESSION USING SVD")
print("="*80)

## 2. Project Overview

In [ ]:
# Project information
project_info = {
    'Project Title': 'Image Compression using Singular Value Decomposition (SVD)',
    'Course': 'Image Analysis and Processing',
    'University': 'KHTN University',
    'Implementation': 'Python 3.8+',
    'Key Libraries': 'NumPy, SciPy, Matplotlib, OpenCV, scikit-image',
    'Notebooks': '5 notebooks (01-05)',
    'Modules': '4 modules (image_utils, svd_compression, quality_metrics, visualization)',
    'Test Images': 'Grayscale (5.3.01.tiff), RGB (4.1.01.tiff)'
}

print("\n" + "="*80)
print("PROJECT INFORMATION")
print("="*80)
for key, value in project_info.items():
    print(f"{key:20s}: {value}")
print("="*80)

## 3. Load Test Images

In [ ]:
# Load images
print("\nLoading test images...")

# Grayscale
gray_path = '../images/grayscale/5.3.01.tiff'
img_gray = load_image(gray_path, mode='GRAY')
print(f"  Grayscale: {gray_path}")
print(f"    Shape: {img_gray.shape}")

# RGB
rgb_path = '../images/color/4.1.01.tiff'
img_rgb = load_image(rgb_path, mode='RGB')
print(f"  RGB: {rgb_path}")
print(f"    Shape: {img_rgb.shape}")

# Display
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title(f'Grayscale Test Image\n{img_gray.shape}', fontsize=13, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_rgb)
axes[1].set_title(f'RGB Test Image\n{img_rgb.shape}', fontsize=13, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('../results/visualizations/final_test_images.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nTest images loaded successfully!")

## 4. Comprehensive Compression Tests

In [ ]:
# Define k values to test
k_values = [5, 10, 20, 30, 50, 70, 100]

print("\n" + "="*80)
print("COMPREHENSIVE COMPRESSION TESTS")
print("="*80)
print(f"k values: {k_values}")
print()

### 4.1 Grayscale Compression

In [ ]:
print("GRAYSCALE COMPRESSION:")
print("-" * 80)
print(f"{'k':>5} | {'PSNR':>8} | {'MSE':>10} | {'SSIM':>8} | {'Saved':>8} | {'Quality'}")
print("-" * 80)

results_gray = {}

for k in k_values:
    # Compress
    compressed = compress_grayscale(img_gray, k)
    
    # Metrics
    metrics = calculate_all_metrics(img_gray, compressed, include_ssim=True)
    stats = calculate_compression_ratio(img_gray.shape, k, is_rgb=False)
    
    # Quality label
    if metrics['psnr'] >= 40:
        quality = 'Excellent'
    elif metrics['psnr'] >= 30:
        quality = 'Good'
    elif metrics['psnr'] >= 20:
        quality = 'Fair'
    else:
        quality = 'Poor'
    
    # Store
    results_gray[k] = {
        'image': compressed,
        'psnr': metrics['psnr'],
        'mse': metrics['mse'],
        'ssim': metrics['ssim'],
        'space_saved': stats['space_saved_percent'],
        'quality': quality
    }
    
    # Print
    print(f"{k:5d} | {metrics['psnr']:8.2f} | {metrics['mse']:10.2f} | {metrics['ssim']:8.4f} | "
          f"{stats['space_saved_percent']:7.1f}% | {quality}")

print("-" * 80)

### 4.2 RGB Compression

In [ ]:
print("\nRGB COMPRESSION:")
print("-" * 80)
print(f"{'k':>5} | {'PSNR':>8} | {'MSE':>10} | {'Saved':>8} | {'Quality'}")
print("-" * 80)

results_rgb = {}

for k in k_values:
    # Compress
    compressed = compress_rgb(img_rgb, k)
    
    # Metrics
    metrics = calculate_all_metrics(img_rgb, compressed, include_ssim=False)
    stats = calculate_compression_ratio(img_rgb.shape, k, is_rgb=True)
    
    # Quality label
    if metrics['psnr'] >= 40:
        quality = 'Excellent'
    elif metrics['psnr'] >= 30:
        quality = 'Good'
    elif metrics['psnr'] >= 20:
        quality = 'Fair'
    else:
        quality = 'Poor'
    
    # Store
    results_rgb[k] = {
        'image': compressed,
        'psnr': metrics['psnr'],
        'mse': metrics['mse'],
        'space_saved': stats['space_saved_percent'],
        'quality': quality
    }
    
    # Print
    print(f"{k:5d} | {metrics['psnr']:8.2f} | {metrics['mse']:10.2f} | "
          f"{stats['space_saved_percent']:7.1f}% | {quality}")

print("-" * 80)

## 5. Summary Statistics

In [ ]:
# Create summary DataFrame
summary_data = {
    'k': k_values,
    'Gray_PSNR': [results_gray[k]['psnr'] for k in k_values],
    'Gray_MSE': [results_gray[k]['mse'] for k in k_values],
    'Gray_SSIM': [results_gray[k]['ssim'] for k in k_values],
    'Gray_Saved%': [results_gray[k]['space_saved'] for k in k_values],
    'RGB_PSNR': [results_rgb[k]['psnr'] for k in k_values],
    'RGB_MSE': [results_rgb[k]['mse'] for k in k_values],
    'RGB_Saved%': [results_rgb[k]['space_saved'] for k in k_values]
}

df_summary = pd.DataFrame(summary_data)

print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)
print(df_summary.to_string(index=False))
print("="*80)

# Save to CSV
df_summary.to_csv('../results/metrics/final_summary.csv', index=False)
print("\nSummary saved to: results/metrics/final_summary.csv")

## 6. Find Optimal k Values

In [ ]:
# Find optimal k (PSNR >= 30dB)
optimal_k_gray = None
optimal_k_rgb = None

for k in k_values:
    if optimal_k_gray is None and results_gray[k]['psnr'] >= 30:
        optimal_k_gray = k
    if optimal_k_rgb is None and results_rgb[k]['psnr'] >= 30:
        optimal_k_rgb = k

if optimal_k_gray is None:
    optimal_k_gray = k_values[-1]
if optimal_k_rgb is None:
    optimal_k_rgb = k_values[-1]

print("\n" + "="*80)
print("OPTIMAL k VALUES (Target: PSNR >= 30dB)")
print("="*80)

print(f"\nGrayscale:")
print(f"  Optimal k: {optimal_k_gray}")
print(f"  PSNR: {results_gray[optimal_k_gray]['psnr']:.2f} dB")
print(f"  SSIM: {results_gray[optimal_k_gray]['ssim']:.4f}")
print(f"  Space saved: {results_gray[optimal_k_gray]['space_saved']:.1f}%")
print(f"  Quality: {results_gray[optimal_k_gray]['quality']}")

print(f"\nRGB:")
print(f"  Optimal k: {optimal_k_rgb}")
print(f"  PSNR: {results_rgb[optimal_k_rgb]['psnr']:.2f} dB")
print(f"  Space saved: {results_rgb[optimal_k_rgb]['space_saved']:.1f}%")
print(f"  Quality: {results_rgb[optimal_k_rgb]['quality']}")

print("="*80)

## 7. Key Visualizations for Report

### 7.1 Comprehensive Comparison Chart

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. PSNR Comparison
x = np.arange(len(k_values))
width = 0.35

axes[0, 0].bar(x - width/2, df_summary['Gray_PSNR'], width, 
               label='Grayscale', color='gray', alpha=0.8)
axes[0, 0].bar(x + width/2, df_summary['RGB_PSNR'], width,
               label='RGB', color='steelblue', alpha=0.8)
axes[0, 0].axhline(y=30, color='red', linestyle='--', linewidth=2, label='Good (30dB)', alpha=0.7)
axes[0, 0].axhline(y=40, color='green', linestyle='--', linewidth=2, label='Excellent (40dB)', alpha=0.7)
axes[0, 0].set_xlabel('k value', fontsize=12)
axes[0, 0].set_ylabel('PSNR (dB)', fontsize=12)
axes[0, 0].set_title('PSNR Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(k_values)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Compression Ratio
axes[0, 1].bar(x - width/2, df_summary['Gray_Saved%'], width,
               label='Grayscale', color='darkgreen', alpha=0.8)
axes[0, 1].bar(x + width/2, df_summary['RGB_Saved%'], width,
               label='RGB', color='orange', alpha=0.8)
axes[0, 1].set_xlabel('k value', fontsize=12)
axes[0, 1].set_ylabel('Space Saved (%)', fontsize=12)
axes[0, 1].set_title('Compression Ratio', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(k_values)
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].set_ylim([0, 105])

# 3. Quality vs k (Grayscale)
ax3 = axes[1, 0]
ax3.plot(k_values, df_summary['Gray_PSNR'], marker='o', linewidth=2, 
         markersize=8, label='PSNR', color='blue')
ax3.set_xlabel('k', fontsize=12)
ax3.set_ylabel('PSNR (dB)', fontsize=12, color='blue')
ax3.tick_params(axis='y', labelcolor='blue')
ax3.grid(True, alpha=0.3)

ax3_twin = ax3.twinx()
ax3_twin.plot(k_values, df_summary['Gray_SSIM'], marker='s', linewidth=2,
              markersize=8, label='SSIM', color='purple')
ax3_twin.set_ylabel('SSIM', fontsize=12, color='purple')
ax3_twin.tick_params(axis='y', labelcolor='purple')
ax3_twin.set_ylim([0, 1.05])

axes[1, 0].set_title('Grayscale: PSNR & SSIM vs k', fontsize=14, fontweight='bold')

# 4. Trade-off: Quality vs Compression
axes[1, 1].plot(df_summary['Gray_Saved%'], df_summary['Gray_PSNR'],
                marker='o', markersize=10, linewidth=2.5, label='Grayscale', color='gray')
axes[1, 1].plot(df_summary['RGB_Saved%'], df_summary['RGB_PSNR'],
                marker='s', markersize=10, linewidth=2.5, label='RGB', color='steelblue')

# Annotate optimal points
axes[1, 1].scatter([results_gray[optimal_k_gray]['space_saved']], 
                   [results_gray[optimal_k_gray]['psnr']],
                   s=300, color='red', marker='*', zorder=5, 
                   label=f'Optimal Gray (k={optimal_k_gray})')
axes[1, 1].scatter([results_rgb[optimal_k_rgb]['space_saved']], 
                   [results_rgb[optimal_k_rgb]['psnr']],
                   s=300, color='darkred', marker='*', zorder=5,
                   label=f'Optimal RGB (k={optimal_k_rgb})')

axes[1, 1].axhline(y=30, color='red', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('Space Saved (%)', fontsize=12)
axes[1, 1].set_ylabel('PSNR (dB)', fontsize=12)
axes[1, 1].set_title('Quality vs Compression Trade-off', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10, loc='lower left')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/visualizations/final_comprehensive_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("Comprehensive analysis chart saved!")

### 7.2 Side-by-Side Comparison at Optimal k

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Row 1: Grayscale
axes[0, 0].imshow(img_gray, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original Grayscale', fontsize=13, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(results_gray[optimal_k_gray]['image'], cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title(f'Compressed (k={optimal_k_gray})\n'
                     f'PSNR={results_gray[optimal_k_gray]["psnr"]:.1f}dB, '
                     f'Saved={results_gray[optimal_k_gray]["space_saved"]:.0f}%',
                     fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

# Error map
error_gray = np.abs(img_gray.astype(np.float64) - results_gray[optimal_k_gray]['image'].astype(np.float64))
im = axes[0, 2].imshow(error_gray, cmap='hot', vmin=0, vmax=30)
axes[0, 2].set_title(f'Error Map\nMax Error={error_gray.max():.1f}', 
                     fontsize=12, fontweight='bold')
axes[0, 2].axis('off')
plt.colorbar(im, ax=axes[0, 2], fraction=0.046, pad=0.04)

# Row 2: RGB
axes[1, 0].imshow(img_rgb)
axes[1, 0].set_title('Original RGB', fontsize=13, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(results_rgb[optimal_k_rgb]['image'])
axes[1, 1].set_title(f'Compressed (k={optimal_k_rgb})\n'
                     f'PSNR={results_rgb[optimal_k_rgb]["psnr"]:.1f}dB, '
                     f'Saved={results_rgb[optimal_k_rgb]["space_saved"]:.0f}%',
                     fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

# Error map (average across channels)
error_rgb = np.mean(np.abs(img_rgb.astype(np.float64) - results_rgb[optimal_k_rgb]['image'].astype(np.float64)), axis=2)
im2 = axes[1, 2].imshow(error_rgb, cmap='hot', vmin=0, vmax=30)
axes[1, 2].set_title(f'Error Map\nMax Error={error_rgb.max():.1f}', 
                     fontsize=12, fontweight='bold')
axes[1, 2].axis('off')
plt.colorbar(im2, ax=axes[1, 2], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.savefig('../results/visualizations/final_optimal_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("Optimal compression comparison saved!")

## 8. Conclusions and Recommendations

In [ ]:
print("\n" + "="*80)
print("CONCLUSIONS AND RECOMMENDATIONS")
print("="*80)

print("\n1. KEY FINDINGS:")
print("-" * 80)
print("   - SVD compression achieves 70-90% space savings")
print("   - Good quality (PSNR >= 30dB) at k = 20-50 for 512x512 images")
print("   - Clear trade-off: higher k → better quality, lower compression")
print("   - Per-channel SVD works well for RGB images")

print("\n2. OPTIMAL k SELECTION:")
print("-" * 80)
print(f"   - Grayscale: k = {optimal_k_gray} (PSNR = {results_gray[optimal_k_gray]['psnr']:.1f}dB, "
      f"Saved = {results_gray[optimal_k_gray]['space_saved']:.0f}%)")
print(f"   - RGB: k = {optimal_k_rgb} (PSNR = {results_rgb[optimal_k_rgb]['psnr']:.1f}dB, "
      f"Saved = {results_rgb[optimal_k_rgb]['space_saved']:.0f}%)")

print("\n3. RECOMMENDATIONS:")
print("-" * 80)
print("   For 512x512 images:")
print("   - High compression (k=10-20): Web thumbnails, previews")
print("   - Balanced (k=20-50): General storage, good quality")
print("   - High fidelity (k=50-100): Archival, medical imaging")
print("")
print("   General rule of thumb:")
print(f"   - k ≈ 0.05-0.10 × min(m,n) for balanced quality/compression")

print("\n4. ADVANTAGES OF SVD COMPRESSION:")
print("-" * 80)
print("   + Mathematically elegant and optimal (Eckart-Young theorem)")
print("   + Simple to understand and implement")
print("   + Flexible k selection for different use cases")
print("   + Good for educational purposes")

print("\n5. LIMITATIONS:")
print("-" * 80)
print("   - Computational complexity O(mn²) - slow for large images")
print("   - Not as efficient as JPEG for natural images")
print("   - Requires storing 3 matrices (U, Σ, V^T)")
print("   - Full-image approach doesn't exploit local correlations")

print("\n6. FUTURE WORK:")
print("-" * 80)
print("   - Implement block-based SVD (like JPEG's 8x8 blocks)")
print("   - Test on larger dataset (20+ images)")
print("   - Compare with JPEG compression")
print("   - Adaptive k selection algorithm")
print("   - YCbCr color space compression")
print("   - GPU acceleration for large images")

print("="*80)

## 9. Export Summary Data for Report

In [ ]:
# Create comprehensive summary report
report_data = {
    'Project Summary': {
        'Total notebooks': 5,
        'Modules implemented': 4,
        'Test images': 2,
        'k values tested': len(k_values),
        'Visualizations created': '30+'
    },
    'Grayscale Results': {
        'Image': '5.3.01.tiff',
        'Dimensions': f'{img_gray.shape[0]}x{img_gray.shape[1]}',
        'Optimal k': optimal_k_gray,
        'Best PSNR': f"{results_gray[optimal_k_gray]['psnr']:.2f} dB",
        'Best SSIM': f"{results_gray[optimal_k_gray]['ssim']:.4f}",
        'Space saved': f"{results_gray[optimal_k_gray]['space_saved']:.1f}%",
        'Quality': results_gray[optimal_k_gray]['quality']
    },
    'RGB Results': {
        'Image': '4.1.01.tiff',
        'Dimensions': f"{img_rgb.shape[0]}x{img_rgb.shape[1]}x{img_rgb.shape[2]}",
        'Optimal k': optimal_k_rgb,
        'Best PSNR': f"{results_rgb[optimal_k_rgb]['psnr']:.2f} dB",
        'Space saved': f"{results_rgb[optimal_k_rgb]['space_saved']:.1f}%",
        'Quality': results_rgb[optimal_k_rgb]['quality']
    }
}

# Save as JSON
import json
with open('../results/metrics/final_report_data.json', 'w') as f:
    json.dump(report_data, f, indent=2)

print("\nReport data exported to: results/metrics/final_report_data.json")

# Print summary
print("\n" + "="*80)
print("FINAL PROJECT SUMMARY")
print("="*80)
for section, data in report_data.items():
    print(f"\n{section}:")
    for key, value in data.items():
        print(f"  {key:20s}: {value}")
print("="*80)

## 10. Files Generated

In [ ]:
print("\n" + "="*80)
print("FILES GENERATED BY THIS PROJECT")
print("="*80)

print("\nSource Code (src/):")
print("  - image_utils.py")
print("  - svd_compression.py")
print("  - quality_metrics.py")
print("  - visualization.py")

print("\nNotebooks (notebooks/):")
print("  - 01_svd_theory_demo.ipynb")
print("  - 02_grayscale_compression.ipynb")
print("  - 03_color_compression.ipynb")
print("  - 04_comparative_analysis.ipynb")
print("  - 05_final_summary.ipynb (this notebook)")

print("\nResults - Compressed Images (results/compressed/):")
print("  - gray_k*.png (grayscale compressed images)")
print("  - color_k*.png (RGB compressed images)")
print("  - rgb_k*.png (RGB compressed images)")

print("\nResults - Visualizations (results/visualizations/):")
print("  - gray_*.png (grayscale analysis charts)")
print("  - rgb_*.png (RGB analysis charts)")
print("  - color_*.png (color analysis charts)")
print("  - comparison_*.png (comparison charts)")
print("  - final_*.png (final summary charts)")

print("\nResults - Metrics (results/metrics/):")
print("  - comparison_gray_vs_rgb.csv")
print("  - final_summary.csv")
print("  - final_report_data.json")

print("\nReport (report/):")
print("  - report_outline.md (detailed report outline)")

print("\nConfiguration:")
print("  - README.md")
print("  - requirements.txt")

print("="*80)
print("\nAll files have been generated successfully!")
print("The project is complete and ready for submission.")
print("="*80)

## Summary

This notebook provides a comprehensive summary of the entire SVD image compression project:

1. **Loaded both test images** (grayscale and RGB)
2. **Ran comprehensive compression tests** with multiple k values
3. **Found optimal k values** for both image types (target: PSNR ≥ 30dB)
4. **Created summary statistics** and exported to CSV/JSON
5. **Generated key visualizations** for the report:
   - Comprehensive comparison charts
   - Trade-off analysis
   - Optimal compression examples
6. **Provided conclusions and recommendations**
7. **Listed all generated files**

### Next Steps for Student:

1. **Fill in the report** using `report/report_outline.md` as template
2. **Insert figures** from `results/visualizations/` into report
3. **Add actual numerical results** from CSV files
4. **Write discussion** based on observations
5. **Prepare presentation** if needed
6. **Submit the project!**

### Resources:
- All code is in `src/`
- All notebooks are in `notebooks/`
- All results are in `results/`
- Report outline is in `report/`
- README.md has full documentation

**Good luck with your presentation! 🎓**